# Using eBFE Models: Spring Creek 2D Analysis

This notebook demonstrates working with FEMA eBFE/BLE models using the Spring Creek (12040102) study area.

## The Problem: eBFE Models Are Broken

**FEMA provides valuable BLE models, but they're intentionally separated into folders that make them UNUSABLE:**

1. **Output/ Separated**: Pre-run HDF results separated from project folder → Can't access results
2. **Terrain/ Misplaced**: Terrain folder outside project → .rasmap references break, model won't run
3. **Absolute DSS Paths**: DSS File= uses paths from original system → "DSS path needs correction" GUI popups

**Without our library**: 30-60 minutes of manual fixes per model (moving folders, correcting paths via GUI dialogs)

**With RasEbfeModels**: One function call → runnable HEC-RAS model with all paths corrected ✓

## Our Solution: 3 Critical Fixes

**RasEbfeModels.organize_spring_creek() automatically**:
1. Moves Output/ HDF files INTO project folder (access pre-run results)
2. Ensures Terrain/ is IN project folder (.rasmap references work)
3. Corrects ALL paths to relative references (no GUI error popups)

**Result**: Model that just works - no manual fixes, no frustration, automation-friendly

## Model Characteristics

- **Pattern 3a**: Single large 2D model with nested zip
- **Size**: 9.7 GB
- **Type**: 2D unsteady flow
- **Plans**: 8 (with pre-computed results)
- **Terrain**: Self-contained, 504.6 MB
- **Version**: HEC-RAS 5.0.7

## What You'll Learn

1. Organize broken eBFE model into runnable HEC-RAS project
2. Understand the 3 critical fixes applied automatically
3. Validate DSS boundary conditions
4. Extract pre-computed 2D results (without re-running)
5. Visualize water surface elevations
6. Optional: Run compute test with haiku validation

## Prerequisites

**Automatic Download**: This notebook will automatically download Spring Creek Models.zip (9.7 GB) from the eBFE S3 bucket if not already present. The download includes:
- Progress tracking with tqdm
- Resume-safe (won't re-download if already present)
- Automatic extraction with progress tracking

**Download Details**:
- **Size**: 9.7 GB
- **Source**: FEMA eBFE S3 bucket
- **Time**: ~10-20 minutes depending on connection speed
- **Disk Space**: ~20 GB required (zip + extracted files)

**Manual Download** (optional, if automatic fails):
1. Visit: https://webapps.usgs.gov/infrm/estBFE/
2. Search for "Spring" study area
3. Download Models.zip (9.7 GB)
4. Extract to desired location

**Important**: Do NOT manually organize the eBFE files - let `RasEbfeModels.organize_spring_creek()` handle it. Manual organization requires extensive path corrections.

In [ ]:
from pathlib import Path
import sys

# Add parent directory to path for development
try:
    from ras_commander import init_ras_project, RasCmdr
except ImportError:
    sys.path.insert(0, str(Path.cwd().parent))
    from ras_commander import init_ras_project, RasCmdr

import matplotlib.pyplot as plt
import pandas as pd

## Step 1: Organize Broken eBFE Model into Runnable HEC-RAS Project

**Automatic Download**: If source data is not present, `organize_spring_creek()` will automatically download 9.7 GB from eBFE S3 bucket. You'll see progress bars for download and extraction.

**RasEbfeModels.organize_spring_creek() applies 3 critical fixes**:

1. **Output/ Integration**: Moves pre-run HDF files into project folder
2. **Terrain/ Integration**: Ensures terrain is in project folder
3. **Path Corrections**: Converts ALL paths to relative references (DSS, terrain, etc.)

**Without these fixes**: Model won't open in HEC-RAS without manual path corrections and folder moves.

**With these fixes**: Model works immediately - no manual intervention required.

In [ ]:
# Import eBFE model organization function
from ras_commander.ebfe_models import RasEbfeModels

# Set paths
downloaded_folder = Path(r"D:\Ras-Commander_BulkData\eBFE\Harris_County\12040102_Spring_Models_extracted")
organized_folder = Path(r"D:\Ras-Commander_BulkData\eBFE\Organized\SpringCreek_12040102")

# Check if already organized
if not organized_folder.exists() or not (organized_folder / "agent" / "model_log.md").exists():
    print("Organizing Spring Creek model...")
    organized_folder = RasEbfeModels.organize_spring_creek(
        downloaded_folder,
        organized_folder,
        validate_dss=True  # Validate DSS boundary conditions
    )
else:
    print(f"Model already organized at: {organized_folder}")

print(f"\n✓ Organized model location: {organized_folder}")

## Understanding the Fixes Applied

### Before RasEbfeModels (Broken eBFE Delivery)

**File Structure** (won't work):
```
12040102_Spring_Models_extracted/
└── 12040102_Models_202207/
    ├── _Final.zip (9.67 GB nested - must extract manually)
    └── _Final_extracted/
        └── _Final/
            └── HECRAS_507/
                ├── Spring.prj ✗ Can't find terrain
                ├── Spring.u01 ✗ DSS File=.\DSS_Input\Spring.dss (wrong path)
                ├── Spring.rasmap ✗ Terrain=.\Terrain\RAS_Terrain\Terrain.hdf (doesn't exist)
                ├── Terrain/ ✓ Exists but in wrong location for .rasmap
                └── Shp/, Features/ (mixed with model files)
```

**User Experience**:
1. Extract nested zip manually (10 minutes)
2. Open Spring.prj → ERROR: "Terrain not found"
3. Try to fix → Realize .rasmap references wrong location
4. Open Spring.prj → ERROR: "DSS path needs correction"
5. Manually fix DSS paths via GUI
6. Try to view results → Can't find HDF files
7. Give up or spend 30+ minutes fixing

### After RasEbfeModels (Runnable HEC-RAS Model)

**File Structure** (works):
```
SpringCreek_12040102/
├── RAS Model/
│   ├── Spring.prj ✓ All paths correct
│   ├── Spring.u01 ✓ DSS File=Spring.dss (relative, exists)
│   ├── Spring.rasmap ✓ Terrain=.\Terrain\Terrain.hdf (correct, exists)
│   ├── Spring.p01.hdf ✓ Pre-run results accessible
│   ├── Spring.dss ✓ In project folder
│   └── Terrain/
│       └── Terrain.hdf ✓ Where .rasmap expects it
├── Spatial Data/ (shapefiles separate from model)
├── Documentation/ (inventory)
└── agent/model_log.md (documents all fixes applied)
```

**User Experience**:
```python
organized = RasEbfeModels.organize_spring_creek(source, validate_dss=True)
init_ras_project(organized / "RAS Model", "5.0.7")
# ✓ Opens without errors
# ✓ Terrain loads
# ✓ DSS files load
# ✓ Pre-run results accessible
# ✓ No manual fixes needed
```

### The 3 Critical Fixes (Automatic)

1. **Terrain Integration**: Terrain/ moved to project folder, .rasmap path corrected
2. **DSS Path Corrections**: All DSS references corrected to relative paths that exist
3. **Output Integration**: Pre-run HDF files in project folder (if present)

**Result**: Model that just works ✓

## Step 2: Verify Organization

Check the standardized 4-folder structure and agent work log.

In [ ]:
# Verify 4-folder structure
folders = ['HMS Model', 'RAS Model', 'Spatial Data', 'Documentation', 'agent']
print("Folder Structure:")
for folder in folders:
    folder_path = organized_folder / folder
    if folder_path.exists():
        file_count = len(list(folder_path.rglob('*')))
        print(f"  ✓ {folder}/ ({file_count} items)")
    else:
        print(f"  ✗ {folder}/ (missing)")

# Check for agent work log
model_log = organized_folder / "agent" / "model_log.md"
if model_log.exists():
    print(f"\n✓ Agent work log: {model_log}")
    print("\nWork log preview (first 20 lines):")
    print("=" * 80)
    print('\n'.join(model_log.read_text().split('\n')[:20]))
    print("=" * 80)

## Step 3: Initialize Project with ras-commander

Initialize the Spring Creek HEC-RAS project using ras-commander.

In [ ]:
# Initialize project
project_folder = organized_folder / "RAS Model"
ras = init_ras_project(project_folder, "5.0.7")

print(f"Project initialized: {ras.prj_file}")
print(f"\nPlans found: {len(ras.plan_df)}")
print(ras.plan_df[['plan_number', 'plan_title', 'plan_file']].to_string())

## Step 4: Validate DSS Boundary Conditions

Spring Creek uses DSS files for boundary conditions. Validate all pathnames.

In [ ]:
from ras_commander.dss import RasDss

# Find DSS files
dss_files = list(project_folder.glob('**/*.dss'))
print(f"Found {len(dss_files)} DSS file(s):\n")

for dss_file in dss_files:
    print(f"Validating: {dss_file.name}")
    
    # Get catalog
    catalog = RasDss.get_catalog(dss_file)
    print(f"  Pathnames: {len(catalog)}")
    
    # Validate each pathname
    invalid_count = 0
    for pathname in catalog['pathname']:
        result = RasDss.check_pathname(dss_file, pathname)
        if not result.is_valid:
            print(f"    ✗ INVALID: {pathname}")
            print(f"      Issue: {result.message}")
            invalid_count += 1
    
    if invalid_count == 0:
        print(f"  ✓ All pathnames valid\n")
    else:
        print(f"  ⚠️ {invalid_count} invalid pathname(s)\n")

## Step 5: Extract Pre-Computed Results

Spring Creek includes pre-computed results for all 8 plans. Extract water surface elevations without re-running.

In [ ]:
from ras_commander.hdf import HdfResultsPlan, HdfMesh

# Extract results from Plan 01
hdf_file = project_folder / "Spring.p01.hdf"
print(f"Reading HDF results: {hdf_file.name}\n")

hdf = HdfResultsPlan(hdf_file)

# Get final time step water surface elevations
wse = hdf.get_wse(time_index=-1)
print(f"Water Surface Elevations:")
print(f"  Count: {len(wse)}")
print(f"  Min: {wse.min():.2f} ft")
print(f"  Max: {wse.max():.2f} ft")
print(f"  Mean: {wse.mean():.2f} ft")

## Step 6: Get 2D Mesh Cell Locations

Extract the 2D mesh cell locations for spatial analysis.

In [ ]:
# Get mesh cell centers
mesh_cells = HdfMesh.get_mesh_cell_points("01", ras_object=ras)

print(f"2D Mesh Cells:")
print(f"  Total cells: {len(mesh_cells)}")
print(f"\nFirst 5 cells:")
print(mesh_cells.head())

## Step 7: Visualize Water Surface Elevations

Plot the water surface elevation spatial distribution.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Combine mesh locations with WSE values
if len(wse) == len(mesh_cells):
    mesh_cells['wse'] = wse
    
    # Create scatter plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    scatter = ax.scatter(
        mesh_cells['cell_center_x'],
        mesh_cells['cell_center_y'],
        c=mesh_cells['wse'],
        cmap='viridis',
        s=1,
        alpha=0.6
    )
    
    plt.colorbar(scatter, ax=ax, label='Water Surface Elevation (ft)')
    ax.set_xlabel('Easting (ft)')
    ax.set_ylabel('Northing (ft)')
    ax.set_title('Spring Creek - Water Surface Elevation (Plan 01, Final Time Step)')
    ax.set_aspect('equal')
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Plotted {len(mesh_cells)} mesh cells")
else:
    print(f"⚠️ Mesh cell count ({len(mesh_cells)}) doesn't match WSE count ({len(wse)})")

## Step 8: Extract Depth and Velocity

Extract additional 2D results from the pre-computed HDF file.

In [ ]:
# Extract depth
depth = hdf.get_depth(time_index=-1)
print(f"Depth:")
print(f"  Count: {len(depth)}")
print(f"  Max: {depth.max():.2f} ft")
print(f"  Mean: {depth.mean():.2f} ft")

# Extract velocity
velocity = hdf.get_velocity(time_index=-1)
print(f"\nVelocity:")
print(f"  Count: {len(velocity)}")
print(f"  Max: {velocity.max():.2f} ft/s")
print(f"  Mean: {velocity.mean():.2f} ft/s")

## Step 9: Compare Multiple Plans

Spring Creek has 8 plans. Extract and compare results across plans.

In [ ]:
# Extract max WSE from each plan
plan_results = []

for plan_num in ['01', '02', '03', '04', '05', '06', '07', '08']:
    hdf_file = project_folder / f"Spring.p{plan_num}.hdf"
    if hdf_file.exists():
        hdf_plan = HdfResultsPlan(hdf_file)
        wse_plan = hdf_plan.get_wse(time_index=-1)
        
        plan_results.append({
            'plan': plan_num,
            'cells': len(wse_plan),
            'wse_min': wse_plan.min(),
            'wse_max': wse_plan.max(),
            'wse_mean': wse_plan.mean()
        })

# Create comparison dataframe
results_df = pd.DataFrame(plan_results)
print("Plan Comparison (Final Time Step):")
print(results_df.to_string(index=False))

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(results_df['plan'], results_df['wse_max'], marker='o', label='Max WSE')
ax.plot(results_df['plan'], results_df['wse_mean'], marker='s', label='Mean WSE')
ax.plot(results_df['plan'], results_df['wse_min'], marker='^', label='Min WSE')
ax.set_xlabel('Plan Number')
ax.set_ylabel('Water Surface Elevation (ft)')
ax.set_title('Spring Creek - WSE Comparison Across Plans')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 10: Check Terrain Configuration

Verify terrain is properly configured (Pattern 3a includes self-contained terrain).

In [ ]:
from ras_commander import RasMap

# Check for .rasmap file
rasmap_files = list(project_folder.glob('*.rasmap'))
if rasmap_files:
    rasmap_file = rasmap_files[0]
    print(f"RAS Mapper file: {rasmap_file.name}")
    
    # Check terrain layer
    terrain_folder = project_folder / "Terrain"
    if terrain_folder.exists():
        terrain_files = list(terrain_folder.glob('*.tif'))
        print(f"\nTerrain files found: {len(terrain_files)}")
        for tf in terrain_files:
            size_gb = tf.stat().st_size / 1e9
            print(f"  - {tf.name}: {size_gb:.2f} GB")
            
            # Validate terrain
            is_valid = RasMap.is_valid_layer(tf, layer_type='terrain')
            print(f"    Valid: {'✓' if is_valid else '✗'}")
    else:
        print("  ⚠️ Terrain folder not found")
else:
    print("⚠️ No .rasmap file found")

## Optional: Compute Test Validation

**Note**: Spring Creek already has pre-computed results. This section shows how to run a compute test to validate terrain/DSS files if needed.

**Skip this section if you just want to use pre-computed results.**

In [ ]:
# Uncomment to run compute test (requires HEC-RAS 5.0.7 installed)
# COMPUTE_TEST = False  # Set to True to run

# if COMPUTE_TEST:
#     print("Running compute test (Plan 01)...")
#     print("This validates terrain, land use, and DSS files are correct.")
#     print("Expected time: 30-60 minutes for 2D model\n")
#     
#     RasCmdr.compute_plan("01", ras_object=ras, num_cores=4)
#     
#     print("\n✓ Compute test complete")
#     print("If plan executed successfully → terrain/DSS files are valid")
# else:
#     print("Compute test skipped (using pre-computed results)")

print("\n💡 Compute test instructions available in:")
compute_instructions = organized_folder / "COMPUTE_TEST_INSTRUCTIONS.md"
if compute_instructions.exists():
    print(f"   {compute_instructions}")
else:
    print("   See agent/model_log.md for compute test command")

## Optional: Check Results with Haiku Subagent

After running a compute test (or using pre-computed results), launch haiku subagent to check for errors/warnings.

In [ ]:
# Check compute messages in HDF
messages = hdf.get_compute_messages()
print("Compute Messages (from pre-computed results):")
print("=" * 80)
print(messages)
print("=" * 80)

# For automated checking, would launch haiku subagent:
# Task(
#     subagent_type="notebook-output-auditor",
#     model="haiku",
#     prompt=f"Check HEC-RAS results in {hdf_file} for errors, warnings, convergence issues"
# )
# Results written to: agent/compute_test_results.md

print("\n💡 For automated error checking, see:")
print("   COMPUTE_TEST_INSTRUCTIONS.md (Test 5 - Haiku Results Check)")

## Summary

This notebook demonstrated:

1. ✓ **Organization**: Used generated `organize_springcreek_12040102()` function
2. ✓ **4-Folder Structure**: HMS/RAS/Spatial/Documentation standardized
3. ✓ **DSS Validation**: Validated boundary condition pathnames
4. ✓ **Results Extraction**: Extracted WSE, depth, velocity from pre-computed results
5. ✓ **2D Visualization**: Plotted spatial water surface elevations
6. ✓ **Terrain Validation**: Verified self-contained terrain files
7. ✓ **Agent Documentation**: agent/model_log.md documents organization

**Pattern 3a Characteristics**:
- Single large 2D model (not multiple streams)
- Nested zip extraction required
- Self-contained terrain (no SpatialData.zip needed)
- Pre-computed results enable immediate analysis

**Next Steps**:
- Extract time series data for specific locations
- Compare results across all 8 plans
- Generate inundation maps
- Export results to GIS formats